In [ ]:
!pip install sklearn
from sklearn.impute import  SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures,MinMaxScaler

# pipline
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

#classification model
from sklearn.neighbors import KNeighborsClassifier

#grid search
from sklearn.model_selection import GridSearchCV, train_test_split

# optional: pipeline visualization
from sklearn import set_config
set_config(display='diagram')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
!pip install tensorflow
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
###

In [ ]:
data=pd.read_csv('https://raw.githubusercontent.com/sebastiancoombs/Kaggle_2022_MM/main/Cleaned%20data/tournament_pm_games (2).csv') ### this should be the plus minues games we created
#normed_games=pd.read_csv('https://raw.githubusercontent.com/sebastiancoombs/Kaggle_2022_MM/main/Cleaned%20data/NCAA_games_normalized_with_rankings.csv')## this is all the games normalized by conference instead of plus minus

InvalidURL: ignored

In [ ]:
data.head(5) ## the teams plus minus scores

In [ ]:
data.columns

In [ ]:
# data=data.drop(data.filter(like='Season').columns,axis=1)
def get_team(day):
  return int(day.split('_')[-1])

def get_season(day):
  return int(day.split('_')[-0])
data.head(5)

In [ ]:
data.Season

# Start building the model pipeline and split the data

In [ ]:
y_test=data[data.Season==2021].Win_Loss
y_train=data[data.Season<2021].Win_Loss
xtest=data[data.Season==2021].drop(['Win_Loss','Season'],axis=1)
xtrain=data[data.Season<2021].drop(['Win_Loss','Season'],axis=1)


In [ ]:
cat_features=list (xtest.filter(like='ConfAbbrev').columns)+ list(xtest.filter(like='team').columns)+list(xtest.filter(like='seed').columns)
cat_features.append('day')
cat_features.
 

In [ ]:
len(y_test)

In [ ]:
ord_features=list (xtest.filter(like='seed').columns)
ord_features

In [ ]:
num_features=[col for col in xtest.columns if col not in ord_features+cat_features]

In [ ]:
num_processor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scaler', StandardScaler())
])
num_processor

In [ ]:
cat_processor = Pipeline(steps=[
    ('cat_imputer', SimpleImputer(strategy='most_frequent')),
    ('cat_encoder',OneHotEncoder(handle_unknown='ignore'))
])
cat_processor

In [ ]:
ord_processor = Pipeline(steps=[
    ('ord_imputer', SimpleImputer(strategy='most_frequent')),
    ('ord_encoder',OrdinalEncoder())
])
ord_processor

In [ ]:
features_processor = ColumnTransformer(transformers = [
    ('num_pipeline', num_processor,num_features),
    ('cat_pipeline', cat_processor,cat_features),
    #('ord_pipeline', ord_processor,ord_features)
],remainder='drop') #drop Name, Sipsp, Parch, Cabin, Ticket
features_processor

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
# classification pipeline
pipe = Pipeline(steps=[
    ('feature_processor', features_processor),
    ('poly_features',PolynomialFeatures()),
    ('classifier',knn)
])
pipe

In [ ]:
# grid search (degree of the poly features, n_neighbors, weights)
param_dic = {'classifier__n_neighbors' : list(range(10,30)),
            'classifier__weights' : ['uniform','distance'],
            'poly_features__degree' : [1,2]
             }
grid = GridSearchCV(pipe,param_dic,cv=5,scoring='accuracy',n_jobs=-1,verbose=1)

In [ ]:
grid.fit(xtrain,y_train)

In [ ]:
pd.DataFrame(grid.cv_results_)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
#best_clf=grid.best_estimator_

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# ypred=best_clf.predict(xtest)
# accuracy_score(ypred,ytest)

In [ ]:
# accuracy_score(ypred,ytest)

In [ ]:
import xgboost as xgb
xgb.XGBClassifier()

In [ ]:
forest_clf= xgb.XGBClassifier()

In [ ]:
pipe2 = Pipeline(steps=[
    ('feature_processor', features_processor),
    ('clf',forest_clf)
])
pipe2

In [ ]:
# grid search (degree of the poly features, n_neighbors, weights)
param_dic2 ={'clf__max_depth':[2,5,10,20],
             'clf__learning_rate':[.01,.05,.1,.2,.3,.4,.5],
              'clf__gamma':[0,.1,.5,.6,.8],
              #'clf__n_estimators':[100,200,400,800],
              #'clf__min_child_weight':[1,2,3,4,5,6,8],
            #  'clf__subsample':[.5,.6,.7,.8,.9,1],
            #  'clf__colsample_bytree':[.5,.7,.8,.9,1],
             }
grid = GridSearchCV(pipe2,param_dic2,cv=5,scoring='accuracy',n_jobs=-1,verbose=1)

In [ ]:
grid.fit(xtrain,y_train)

In [ ]:
grid.best_params_


In [ ]:
grid.best_score_

In [ ]:
best_clf=grid.best_estimator_

Submissions are scored on the log loss:

$$
LogLoss=\frac{-1}{n}\sum_{𝑖=1}^{𝑛}[𝑦𝑖log(𝑦̂ 𝑖)+(1−𝑦𝑖)log(1−𝑦̂ 𝑖)],
$$
where

𝑛 is the number of games played
𝑦̂ 𝑖 is the predicted probability of team 1 beating team 2
𝑦𝑖 is 1 if team 1 wins, 0 if team 2 wins
𝑙𝑜𝑔 is the natural logarithm

The use of the logarithm provides extreme punishments for being both confident and wrong. In the worst possible case, a prediction that something is true when it is actually false will add an infinite amount to your error score. In order to prevent this, predictions are bounded away from the extremes by a small value.



In [ ]:
ypred=best_clf.predict(xtest)
yproba=best_clf.predict_proba(xtest)
yproba=np.array(yproba)
accuracy_score(ypred,y_test)

In [ ]:
len(ypred)

In [ ]:
def logloss(y,yhat):
  n=len(y)
  z=y*np.log(yhat)+(1-y)*np.log(1-yhat,axis=1)
  loss=(-1/n)*np.sum(z)
  return loss

Submission File
The file you submit will depend on whether the competition is in stage 1--historical model building--or stage 2--the 2022 tournament. Sample submission files will be provided for both stages. The format is a list of every possible matchup between the tournament teams. Since team1 vs. team2 is the same as team2 vs. team1, we only include the game pairs where team1 has the lower team id. For example, in a tournament of 68 teams, you will predict (68∗67)/2=2,278 matchups.

Each game has a unique id created by concatenating the season in which the game was played, the team1 id, and the team2 id. For example, "2016_1107_1110" indicates team 1107 potentially played team 1110 in the year 2016. You must predict the probability that the team with the lower id beats the team with the higher id.

The resulting submission format looks like the following, where "pred" represents the predicted probability that the first team will win:

ID,Pred\
2016_1107_1110,0.5\
2016_1107_1112,0.5\
2016_1107_1113,0.5

In [ ]:
from itertools import combinations

In [ ]:
conferences=pd.read_csv('https://raw.githubusercontent.com/sebastiancoombs/Kaggle_2022_MM/main/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneySeeds.csv')
conferences.head(5)

In [ ]:
plus_minus=pd.read_csv('https://raw.githubusercontent.com/sebastiancoombs/Kaggle_2022_MM/main/Cleaned%20data/NCAA_teams_plus_minus.csv')

In [ ]:
teams=[c for c in conferences.loc[conferences.Season==1985].TeamID.unique()]

In [ ]:
plus_minus

In [ ]:
test=pd.DataFrame(list(combinations(teams,2)))

test['season']=1985
test.head()

In [ ]:
season_list=[]
for season in conferences.Season.unique():
  teams=[c for c in conferences.loc[conferences.Season==season].TeamID.unique()]
  combo=list(combinations(teams,2))
  data=pd.DataFrame(data=combo,columns=['team1','team2'])
  data['Season']=season
  season_list.append(data)
sub_file=pd.concat(season_list)
sub_file

In [ ]:
sub_file.Season.unique()

In [ ]:
yrs=[*range(1985,2003)]

sub_file=sub_file[sub_file.Season.isin(yrs)==False]

In [ ]:
sub_file

In [ ]:
sub_file['GameID']= sub_file.Season.astype(str)+'_'+sub_file.team1.astype(str)+'_'+sub_file.team2.astype(str)
sub_file['team1'] = sub_file.Season.astype(str)+'_'+sub_file.team1.astype(str)
sub_file['team2'] = sub_file.Season.astype(str)+'_'+sub_file.team2.astype(str)

sub_file.head(3)

In [ ]:
pm = pd.read_csv('https://raw.githubusercontent.com/sebastiancoombs/Kaggle_2022_MM/main/Cleaned%20data/NCAA_teams_plus_minus.csv')
pm.columns

In [ ]:
# we need to import pm (plus minus) data frame in order to merge 
Wtest=pd.merge(sub_file,pm,left_on='team1',right_on='TeamID')
Wtest



In [ ]:
Ltest=pd.merge(Wtest,pm,left_on='team2',right_on='TeamID')

In [ ]:
test_data = Ltest.drop(['Season_x','Unnamed: 0_x', 'TeamID_x','TeamID_y','Season_y', 'Unnamed: 0_y'],axis=1)
test_data

In [ ]:
test_data.rename(columns = {'team1':'Wteam'}, {'team2':'Lteam'}, {'Seed_x':'Wseed'}, {'Seed_y':'Lseed'} inplace = True)

In [ ]:
test_data.columns

In [ ]:
#Index(['day', 'Wteam', 'Lteam', 'Wseed', 'Lseed', 'Win_Loss', 'Ast_x', 'Blk_x',
      # 'DR_x', 'DayNum_x', 'FGA_x', 'FGA3_x', 'FGM_x', 'FGM3_x', 'FTA_x',
      # 'FTM_x', 'Loc_x', 'OR_x', 'PF_x', 'Rank_day_x', 'Score_x', 'Stl_x',
      # 'TO_x', 'ConfAbbrev_x', 'Ast_y', 'Blk_y', 'DR_y', 'DayNum_y', 'FGA_y',
      # 'FGA3_y', 'FGM_y', 'FGM3_y', 'FTA_y', 'FTM_y', 'Loc_y', 'OR_y', 'PF_y',
      # 'Rank_day_y', 'Score_y', 'Stl_y', 'TO_y', 'ConfAbbrev_y', 'Season'],
      #dtype='object'

In [ ]:
win_proba = best_clf.predict_proba(test_data)

In [ ]:
y_test=data[data.Season==2021].Win_Loss
x_test=data[data.Season==2021].drop(['Win_Loss','Season'],axis=1)


In [ ]:
cat_features=list (xtest.filter(like='ConfAbbrev').columns)+ list(xtest.filter(like='team').columns)+list(xtest.filter(like='seed').columns)
cat_features.append('day')
cat_features

In [ ]:
ord_features=list (xtest.filter(like='seed').columns)
ord_features

In [ ]:
num_features=[col for col in xtest.columns if col not in ord_features+cat_features]

In [ ]:
num_processor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scaler', StandardScaler())
])
num_processor

In [ ]:
cat_processor = Pipeline(steps=[
    ('cat_imputer', SimpleImputer(strategy='most_frequent')),
    ('cat_encoder',OneHotEncoder(handle_unknown='ignore'))
])
cat_processor

In [ ]:
ord_processor = Pipeline(steps=[
    ('ord_imputer', SimpleImputer(strategy='most_frequent')),
    ('ord_encoder',OrdinalEncoder())
])
ord_processor

In [ ]:
features_processor = ColumnTransformer(transformers = [
    ('num_pipeline', num_processor,num_features),
    ('cat_pipeline', cat_processor,cat_features),
    #('ord_pipeline', ord_processor,ord_features)b
],remainder='drop') #drop Name, Sipsp, Parch, Cabin, Ticket
features_processor

In [ ]:
ypred=best_clf.predict(xtest)
yproba=best_clf.predict_proba(xtest)
yproba=np.array(yproba)
accuracy_score(ypred,y_test)